Hello! I fit a random forest regressor as a baseline model for the March 2022 TPS (Traffic congestion prediction).

In [ ]:
N_ESTIMATORS = 400 # Number of estimators of RandomForestRegressor

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
sample = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/sample_submission.csv')
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/test.csv')

train.head()

Check for duplicates and missing values (there are none):

In [ ]:
print(sum(train.duplicated()))
print(sum(test.duplicated()))

In [ ]:
print(np.sum(train.isnull(),0))
print(np.sum(test.isnull(),0))

Process datetime column for training and test set (I should clean this up and not repeat code!):

In [ ]:
train['year'] = train['time'].map(lambda x: int(x[:4]))
train['month'] = train['time'].map(lambda x: int(x[5:7]))
train['day'] = train['time'].map(lambda x: int(x[9:11]))
train['hour'] = train['time'].map(lambda x: int(x[11:13]))
train['minute'] = train['time'].map(lambda x: int(x[14:16]))
train['second'] = train['time'].map(lambda x: int(x[17:19]))
train.describe().T

In [ ]:
test['year'] = test['time'].map(lambda x: int(x[:4]))
test['month'] = test['time'].map(lambda x: int(x[5:7]))
test['day'] = test['time'].map(lambda x: int(x[9:11]))
test['hour'] = test['time'].map(lambda x: int(x[11:13]))
test['minute'] = test['time'].map(lambda x: int(x[14:16]))
test['second'] = test['time'].map(lambda x: int(x[17:19]))
test.describe().T

Encode direction variable and drop rows to get X, y as training set, and Xtest

In [ ]:
from sklearn.preprocessing import OneHotEncoder
one_dir = OneHotEncoder()
direction_encoded = one_dir.fit_transform(np.array(train['direction']).reshape(-1,1)).todense()
X = pd.concat((train, pd.DataFrame(direction_encoded)), axis=1)
print(X.describe().T)
X = X.drop(['row_id','congestion','time','direction'],axis=1)
y = train['congestion']

test_direction_encoded = one_dir.transform(np.array(test['direction']).reshape(-1,1)).todense()
Xtest = pd.concat((test, pd.DataFrame(test_direction_encoded)), axis=1)
print(Xtest.describe().T)
Xtest = Xtest.drop(['row_id','time','direction'],axis=1)
print(Xtest.describe().T)

Simple 85%/15% training/test split. Fit a Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
Xtrain, Xvalid, ytrain, yvalid = train_test_split(X, y, test_size=0.15, random_state=1)
import time
rfr = RandomForestRegressor(n_estimators = N_ESTIMATORS)
print(Xtrain.shape)
print(ytrain.shape)
print('Fitting Random Forest Regressor...')
tt = time.time()
rfr.fit(Xtrain, ytrain)
print('Done in %.1fs' % (time.time() - tt))

In [ ]:
yvalid_pred = rfr.predict(Xvalid)
from sklearn.metrics import mean_absolute_error as MAE
print('Mean absolute error on validation data: %.2f'%MAE(yvalid, yvalid_pred))

Predict using test data and submit!

In [ ]:
predictions = rfr.predict(Xtest)
print(predictions)

output = pd.DataFrame({'row_id': test.row_id, 'congestion': predictions})
output.to_csv('submission.csv', index=False)